### Imports and Environemnt

In [67]:
# Authors: Curtis McGinity
# Emails: mcginity@stanford.edu
# Affiliation: Stanford University Medical Center - Snyder Lab - Genetics Department
# License:
# Creative Commons Attribution Non Commercial 4.0 International : https://creativecommons.org/licenses/by-nc/4.0/legalcode


### GOOGLE CLOUD SETUP ###
# from google.colab import auth
# auth.authenticate_user()

# import google.cloud as gcp
# import gcsfs
from google.cloud import storage


# from google.colab import drive
# drive.mount('/content/drive')

# from google.colab import data_table
# data_table.enable_dataframe_formatter()

# from google.colab.data_table import DataTable
# DataTable.max_columns = 50

### TOOLS ###
import os
from datetime import datetime, timedelta
import time
import sys
import shutil
import pytz


### DATA SCIENCE ###
import pandas as pd
import numpy as np
# import scipy as sp

### DATA VISUALIZATION ###
# import matplotlib
# import matplotlib.pyplot as plt
# import seaborn as sns
# import bokeh
from bokeh.models import ColumnDataSource, LinearColorMapper, widgets, CustomJS
# from bokeh.models.widgets import Slider
from bokeh.io import push_notebook
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, column
from ipywidgets import interact

### REDCAP API
#todo: add redcap api


### ### ### ### ### ### ### ### ### ### ### ###



### HOUSEKEEPING
### project directories
proj_name =  "HIITvsMICT"
pproj = "/content/drive/MyDrive/research/proj/HIITvsMICT/Data"

# Print package versions for reproducibility
print(f"Python version: {sys.version}")
# print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
# print(f"Matplotlib version: {matplotlib.__version__}")
# print(f"Seaborn version: {sns.__version__}")
# print(f"SciPy version: {sp.__version__}")
# print(f"Scikit-learn version: {sklearn.__version__}")

# Print current date and time for notebook reproducibility
print(f"Notebook executed on {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

print('Packages loaded and environment initialized...')

# Dataset ingest
#todo: loop through list_buckets(), extract metadata


Python version: 3.10.12 (main, Jul  5 2023, 15:02:25) [Clang 14.0.6 ]
Pandas version: 1.5.3
Notebook executed on 2023-09-28 13:14:09
Packages loaded and environment initialized...


### Functions and Parameters

In [68]:
output_notebook()

Loading BokehJS ...

### Demo data

In [70]:
# Generate random time series data
dates = [(datetime.now() - timedelta(days=i)) for i in range(10)][::-1]
values = np.random.randint(1, 10, size=len(dates))

time_series_data = {
    'dates': dates,
    'values': values
}

# Generate random heatmap data
heatmap_data = {
    'x': list(range(10)),
    'y': list(range(10)),
    'values': np.random.randint(0, 100, size=(10, 10)).flatten()
}

# Create data sources
time_series_source = ColumnDataSource(time_series_data)
time_series_source_orig = ColumnDataSource(time_series_data)
heatmap_source = ColumnDataSource(heatmap_data)

#### Time Series

In [71]:
time_series_plot = figure(width=600, height=250, x_axis_type='datetime', title="Time Series Data")
time_series_plot.line(x='dates', y='values', source=time_series_source)
show(time_series_plot)

#### Heatmap

In [72]:
heatmap_plot = figure(width=400, height=400, title="Heatmap")
colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
mapper = LinearColorMapper(palette=colors, low=heatmap_data['values'].min(), high=heatmap_data['values'].max())
heatmap_plot.rect(x="x", y="y", width=1, height=1, source=heatmap_source, fill_color={'field': 'values', 'transform': mapper})

GlyphRenderer(id='p2148', ...)

#### Interactive Plot

In [76]:

def update(attr, old, new):
    threshold = new
    filtered_dates = [date for date, value in zip(dates, values) if value > threshold]
    filtered_values = [value for value in values if value > threshold]
    print(f'dates: {len(filtered_dates)}')
    print(f'values: {len(filtered_values)}')
    
    time_series_source.data = {'dates': filtered_dates, 'values': filtered_values}
    print(f'ts: {time_series_source.data}')
    push_notebook(handle=handle)


# Define a JavaScript callback for the slider
callback = CustomJS(args=dict(source=time_series_source, source_orig = time_series_source_orig), code="""
    var data = source.data;
    var data_orig = source_orig.data;
                    
    var threshold = cb_obj.value;
    var values = data['values'];
    var values_orig = data_orig['values']
    for (var i = 0; i < values_orig.length; i++) {
        if (values_orig[i] < threshold)
            values[i] = null;
        else
            values[i] = values_orig[i];
    }
    source.change.emit();
""")
threshold_slider = widgets.Slider(start=0, end=10, value=5, step=1, title="Threshold")
# threshold_slider.on_change('value', update)
threshold_slider.js_on_change('value', callback)

layout = column(threshold_slider, row(time_series_plot, heatmap_plot))
handle = show(layout, notebook_handle=True)

# interact(update, threshold=(0,10,1))